In [ ]:
import sys

sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

# import data.drawings.make_tasks as drawing_tasks
from src.analysis_utilities import SynthesisExperimentAnalyzer
from src.config_builder import ExperimentType

In [ ]:
sns.set_theme(style = "whitegrid", font_scale=1.25, rc={'figure.figsize':(12, 8)})

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
#for random vs cosine similar
# EXPERIMENT_NAME = "compare_lilo_synthetic"

#for human vs synthetic
EXPERIMENT_NAME = "runs_multi"

COMPUTE_LIKELIHOODS = True

FIGURES_DIR = os.path.join("figures", EXPERIMENT_NAME)
os.makedirs(FIGURES_DIR, exist_ok=True)

In [ ]:
# if random vs cos similar, choose cos_similar and romdon
# if human vs synthesis, choose everything else

analyzer = SynthesisExperimentAnalyzer(
    experiment_name=EXPERIMENT_NAME,
    experiment_types=[
        "baseline_dreamcoder", 
        "gpt_solver", 
        "gpt_solver_search", 
        "gpt_solver_stitch_namer_search"
        # "cos_similar",
        # "random",
    ],
    allow_incomplete_results=True,
    compute_likelihoods=COMPUTE_LIKELIHOODS,
    batch_size = 96,
    seeds = [111, 222, 333],
)

In [ ]:
df_results = analyzer.get_synthesis_summary()
df_results = analyzer.format_dataframe_camera(df_results)
df_results

In [ ]:
# get get results for human and synthetic data
df_results = df_results[df_results['Split'] == 'test']

df_results['language'] = df_results['domain'].apply(lambda x: "human" if "human" in x else "synthetic")
df_results['domain'] = df_results['domain'].apply(lambda x: x.replace("(human)", "").strip())


df_results

In [ ]:
# plot random vs cos similar
analyzer.EXPERIMENT_TYPES_PALETTE['Random'] =  '#e56b6f'  # replace 'color_for_synthetic' with actual color
analyzer.EXPERIMENT_TYPES_PALETTE['Cosine Similarity'] = '#0047AB'  # replace 'color_for_synthetic' with actual color
with sns.plotting_context("poster", font_scale=1):
    g = sns.relplot(
        data=df_results,
        kind="line",
        col="domain",
        x="Iteration",
        y=analyzer.COL_NAMES_CAMERA["percent_solved"],
        hue = "Model",
        style="Split",
        markers=[".", "*"],
        markersize=20,
        col_order = [d for d in list(analyzer.DOMAIN_NAMES_CAMERA.values()) if d in list(df_results["domain"])],
        facet_kws={'sharex': False, 'sharey': True, 'margin_titles': True},
        palette=analyzer.EXPERIMENT_TYPES_PALETTE,
        legend=True,
    )
    for subplot in g.axes[0]:
        subplot.set_title(subplot.title.get_text().split(" = ")[-1])
        
    g.axes[0][0].set_xticks(np.arange(0, 16, 3))
    g.axes[0][1].set_xticks(np.arange(0, 10, 3))
    g.axes[0][2].set_xticks(np.arange(0, 10, 3))
        
    # sns.move_legend(g, "best")
    # legend_labels = ['Random', 'Cosine Similarity']
    # legend_colors = ['#40E0D0', '#FF6347']  # The colors you've chosen for 'synthetic' and 'human'
    # legend_patches = [matplotlib.patches.Patch(color=c, label=l) for c, l in zip(legend_colors, legend_labels)]
    # plt.legend(handles=legend_patches, title='Sampling Method', loc='best')
    sns.move_legend(g, loc="lower center", ncol=3, bbox_to_anchor=(0.5, -0.1))

    plt.savefig(os.path.join(FIGURES_DIR, "main_results.pdf"), dpi=300, bbox_inches="tight")

In [ ]:
# plot human vs synthetic language 
import matplotlib
analyzer.EXPERIMENT_TYPES_PALETTE['synthetic'] =  '#40E0D0'  
analyzer.EXPERIMENT_TYPES_PALETTE['human'] = '#FF6347'  
with sns.plotting_context("poster", font_scale=1):
    g = sns.relplot(
        data=df_results,
        kind="line",
        row="domain",
        col=analyzer.COL_NAMES_CAMERA["experiment_type"],
        x="Iteration",
        y=analyzer.COL_NAMES_CAMERA["percent_solved"],
        hue = "language",
        style="Split",
        markers=[".", "*"],
        markersize=20,
        row_order = [d for d in list(analyzer.DOMAIN_NAMES_CAMERA.values()) if d in list(df_results["domain"])],
        facet_kws={'sharex': False, 'sharey': True, 'margin_titles': True},
        palette=analyzer.EXPERIMENT_TYPES_PALETTE,
        legend=True,
    )
    for subplot in g.axes[0]:
        subplot.set_title(subplot.title.get_text().split(" = ")[-1])
        
    for ax in g.axes[0]:
        ax.set_xticks(np.arange(0, 16, 3))
    for ax in g.axes[1]:
        ax.set_xticks(np.arange(0, 10, 3))
    for ax in g.axes[2]:
        ax.set_xticks(np.arange(0, 10, 3))
        
    sns.move_legend(g, loc="lower center", ncol=3, bbox_to_anchor=(0.5, -0.1))
    # legend_labels = ['synthetic', 'human']
    # legend_colors = ['#40E0D0', '#FF6347']  
    # legend_patches = [matplotlib.patches.Patch(color=c, label=l) for c, l in zip(legend_colors, legend_labels)]
    # plt.legend(handles=legend_patches, title='language', loc='best')

    plt.savefig(os.path.join(FIGURES_DIR, "main_results.pdf"), dpi=300, bbox_inches="tight")